In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
IMAGE_SIZE = 128

In [ ]:
file1 = "Cat_Dog_Train_Data_128X128_1.npy"
file2 = "Cat_Dog_Train_Data_128X128_2.npy"
file3 = "Cat_Dog_Train_Data_128X128_3.npy"
file4 = "Cat_Dog_Train_Data_128X128_4.npy"
file6 = "Cat_Dog_Train_Data_128X128_5.npy"
file5 = "Cat_Dog_Train_Data_128X128_6.npy"
#path = f"/content/gdrive/My Drive/models/dogs-vs-cats/{file_name}"

In [ ]:
train_data_1 = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file1}", allow_pickle=True)
train_data_2 = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file2}", allow_pickle=True)
train_data_3 = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file3}", allow_pickle=True)
train_data_4 = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file4}", allow_pickle=True)
test_data = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file6}", allow_pickle=True)
test_data_1 = np.load(f"/content/gdrive/My Drive/models/dogs-vs-cats/{file5}", allow_pickle=True)

In [ ]:
train_data = np.concatenate((train_data_1, train_data_2, train_data_3, train_data_4))
train_data.shape

(2000, 2)

In [ ]:
test_data.shape

(500, 2)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense,Input,Flatten, BatchNormalization,Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import random
import matplotlib.pyplot as plt

In [ ]:
X_train = np.array([i[0] for i in train_data])
X_train = X_train.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y_train = np.array([i[1] for i in train_data])


X_test = np.array([i[0] for i in test_data])
X_test = X_test.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y_test = np.array([i[1] for i in test_data])

# #test_data_1 test_data_1
X_test_1 = np.array([i[0] for i in test_data_1])
X_test_1 = X_test_1.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y_test_1 = np.array([i[1] for i in test_data_1])

In [ ]:
X_train.shape
X_test.shape

(500, 128, 128, 3)

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))

In [ ]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
model = Sequential()
#model.add(Conv2D(3, (3, 3), activation='relu', input_shape=X_train[0].shape))
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

In [ ]:
X_train[0].shape

(128, 128, 3)

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 32, 32, 256)     

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
#model.add(Dense(units=2, activation='softmax'))

In [ ]:
#model.add(conv2d())
model.add(Flatten())
model.add(Dense(2, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 32, 32, 256)     

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])#categorical_crossentropy

In [ ]:
model.fit(x=X_train, y= y_train, batch_size=1000, epochs=25, verbose=2, validation_data=(X_test, y_test))

Epoch 1/25
2/2 - 253s - loss: 0.6948 - accuracy: 0.5040 - val_loss: 0.6901 - val_accuracy: 0.4880
Epoch 2/25
2/2 - 249s - loss: 0.6816 - accuracy: 0.5465 - val_loss: 0.6738 - val_accuracy: 0.6400
Epoch 3/25
2/2 - 254s - loss: 0.6661 - accuracy: 0.6590 - val_loss: 0.6592 - val_accuracy: 0.6600
Epoch 4/25
2/2 - 256s - loss: 0.6506 - accuracy: 0.6750 - val_loss: 0.6441 - val_accuracy: 0.6900
Epoch 5/25
2/2 - 256s - loss: 0.6324 - accuracy: 0.7210 - val_loss: 0.6292 - val_accuracy: 0.7540
Epoch 6/25
2/2 - 261s - loss: 0.6142 - accuracy: 0.7800 - val_loss: 0.6165 - val_accuracy: 0.7580
Epoch 7/25
2/2 - 251s - loss: 0.5981 - accuracy: 0.8080 - val_loss: 0.6030 - val_accuracy: 0.7740
Epoch 8/25
2/2 - 255s - loss: 0.5819 - accuracy: 0.8140 - val_loss: 0.5891 - val_accuracy: 0.7880
Epoch 9/25
2/2 - 255s - loss: 0.5662 - accuracy: 0.8180 - val_loss: 0.5763 - val_accuracy: 0.7940
Epoch 10/25
2/2 - 257s - loss: 0.5519 - accuracy: 0.8210 - val_loss: 0.5646 - val_accuracy: 0.8040
Epoch 11/25
2/2 - 2

In [ ]:
import os
path = "/content/gdrive/My Drive/models/cat_vs_dog_classifier_128X128.h5"
if os.path.isfile(path) is False:
  model.save(path)

In [ ]:
from tensorflow.keras.models import load_model
path = "/content/gdrive/My Drive/models/cat_vs_dog_classifier_128X128.h5"
new_model = load_model(path)

In [ ]:
predictions = new_model.predict(x=X_test_1, verbose=0)

In [ ]:
predictions[:5]

array([[0.2752476 , 0.72475237],
       [0.52680767, 0.47319236],
       [0.59550464, 0.40449536],
       [0.564457  , 0.435543  ],
       [0.231743  , 0.7682571 ]], dtype=float32)

In [ ]:
cm = confusion_matrix(np.argmax(y_test_1, axis=-1), y_pred=np.argmax(predictions, axis=-1))

In [ ]:
cm

array([[208,  33],
       [ 51, 208]])